In [ ]:
train['label'] = train['label'].str.split('/').str[0]
train.head()

,filename,label
0,/kaggle/input/leaf-disease-classification/trai...,7
1,/kaggle/input/leaf-disease-classification/trai...,7
2,/kaggle/input/leaf-disease-classification/trai...,7
3,/kaggle/input/leaf-disease-classification/trai...,7
4,/kaggle/input/leaf-disease-classification/trai...,7


In [ ]:
Id=[]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/leaf-disease-classification/test'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
Id[:5]

['/kaggle/input/leaf-disease-classification/test/7/gkaopa30b7stwoxo6ady.jpg',
 '/kaggle/input/leaf-disease-classification/test/7/c3rxo3tdjelm7a9athn2.jpg',
 '/kaggle/input/leaf-disease-classification/test/7/nfr1v5nncc5qafn0felq.jpg',
 '/kaggle/input/leaf-disease-classification/test/7/cnugacybapknatskv28o.jpg',
 '/kaggle/input/leaf-disease-classification/test/7/kvdh4mr74vgvo2hqibaq.jpg']

In [ ]:
test=pd.DataFrame()
test=test.assign(filename=Id)
test.head()

,filename
0,/kaggle/input/leaf-disease-classification/test...
1,/kaggle/input/leaf-disease-classification/test...
2,/kaggle/input/leaf-disease-classification/test...
3,/kaggle/input/leaf-disease-classification/test...
4,/kaggle/input/leaf-disease-classification/test...


In [7]:
test['label']=test['filename']
test['label']=test['label'].str.replace('/kaggle/input/leaf-disease-classification/test/','')
test.head()

,filename,label
0,/kaggle/input/leaf-disease-classification/test...,7/gkaopa30b7stwoxo6ady.jpg
1,/kaggle/input/leaf-disease-classification/test...,7/c3rxo3tdjelm7a9athn2.jpg
2,/kaggle/input/leaf-disease-classification/test...,7/nfr1v5nncc5qafn0felq.jpg
3,/kaggle/input/leaf-disease-classification/test...,7/cnugacybapknatskv28o.jpg
4,/kaggle/input/leaf-disease-classification/test...,7/kvdh4mr74vgvo2hqibaq.jpg


In [ ]:
test['label'] = test['label'].str.split('/').str[0]
test.head()

,filename,label
0,/kaggle/input/leaf-disease-classification/test...,7
1,/kaggle/input/leaf-disease-classification/test...,7
2,/kaggle/input/leaf-disease-classification/test...,7
3,/kaggle/input/leaf-disease-classification/test...,7
4,/kaggle/input/leaf-disease-classification/test...,7


In [9]:
import tensorflow as tf
import numpy as np 
from PIL import Image

model = tf.saved_model.load('/kaggle/input/resnet50-leaf-disease-classification/ResNet50')
classes = [ "0" ,  "1" ,  "2" ,  "3" ,  "4" ,  "5" ,  "6" ,  "7" ,  "8" ,  "9" , ]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
result=[]
for i in test.filename:
    img = Image.open(i).convert('RGB')
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)
    inp_numpy = np.array(img)[None]
    inp = tf.constant(inp_numpy, dtype='float32')
    class_scores = model(inp)[0].numpy()
    result.append(classes[class_scores.argmax()])
result[:5]

/tmp/ipykernel_20/564266034.py:4: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)


['0', '7', '1', '8', '7']

In [11]:
test=test.assign(prediction=result)
test.head()

,filename,label,prediction
0,/kaggle/input/leaf-disease-classification/test...,7,0
1,/kaggle/input/leaf-disease-classification/test...,7,7
2,/kaggle/input/leaf-disease-classification/test...,7,1
3,/kaggle/input/leaf-disease-classification/test...,7,8
4,/kaggle/input/leaf-disease-classification/test...,7,7


In [12]:
result=[]
for i in train.filename:
    img = Image.open(i).convert('RGB')
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)
    inp_numpy = np.array(img)[None]
    inp = tf.constant(inp_numpy, dtype='float32')
    class_scores = model(inp)[0].numpy()
    result.append(classes[class_scores.argmax()])
result[:5]

/tmp/ipykernel_20/1967585974.py:4: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)


['7', '7', '7', '0', '1']

In [13]:
train=train.assign(prediction=result)
train.head()

,filename,label,prediction
0,/kaggle/input/leaf-disease-classification/trai...,7,7
1,/kaggle/input/leaf-disease-classification/trai...,7,7
2,/kaggle/input/leaf-disease-classification/trai...,7,7
3,/kaggle/input/leaf-disease-classification/trai...,7,0
4,/kaggle/input/leaf-disease-classification/trai...,7,1


In [14]:
from sklearn.metrics import classification_report
print(classification_report(train['label'],train['prediction']))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1215
           1       0.91      0.93      0.92       575
           2       0.96      0.99      0.97       902
           3       0.91      0.99      0.95      1068
           4       0.99      1.00      0.99      2975
           5       0.84      0.99      0.91       542
           6       0.96      0.91      0.93       214
           7       0.99      0.84      0.91       989
           8       0.98      0.82      0.89       791
           9       0.97      0.93      0.94       898

    accuracy                           0.95     10169
   macro avg       0.94      0.94      0.94     10169
weighted avg       0.96      0.95      0.95     10169



In [15]:
print(classification_report(test['label'],test['prediction']))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       304
           1       0.86      0.87      0.86       144
           2       0.94      0.99      0.97       225
           3       0.89      0.96      0.92       267
           4       0.98      0.99      0.99       744
           5       0.77      0.99      0.87       136
           6       0.94      0.89      0.91        54
           7       0.99      0.81      0.89       247
           8       0.97      0.74      0.84       198
           9       0.94      0.91      0.92       224

    accuracy                           0.93      2543
   macro avg       0.92      0.91      0.91      2543
weighted avg       0.94      0.93      0.93      2543



In [16]:
sol=pd.read_csv('/kaggle/input/leaf-disease-classification/submission.csv')
sol.head()

,Image
0,kc0yq3aoedc1b0v05gwp.jpg
1,udxt71dqhpo3je6alnu3.jpg
2,n05xwwbvd8t0k2s6vxim.jpg
3,fe2udf0166l7e914798x.jpg
4,royn55uoa0oc330cmivt.jpg


In [17]:
sol['filename']='/kaggle/input/leaf-disease-classification/sol/'+sol['Image']
sol.head()

,Image,filename
0,kc0yq3aoedc1b0v05gwp.jpg,/kaggle/input/leaf-disease-classification/sol/...
1,udxt71dqhpo3je6alnu3.jpg,/kaggle/input/leaf-disease-classification/sol/...
2,n05xwwbvd8t0k2s6vxim.jpg,/kaggle/input/leaf-disease-classification/sol/...
3,fe2udf0166l7e914798x.jpg,/kaggle/input/leaf-disease-classification/sol/...
4,royn55uoa0oc330cmivt.jpg,/kaggle/input/leaf-disease-classification/sol/...


In [18]:
result=[]
for i in sol.filename:
    img = Image.open(i).convert('RGB')
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)
    inp_numpy = np.array(img)[None]
    inp = tf.constant(inp_numpy, dtype='float32')
    class_scores = model(inp)[0].numpy()
    result.append(classes[class_scores.argmax()])
result[:5]

/tmp/ipykernel_20/3826586461.py:4: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)


['7', '0', '4', '3', '4']

In [19]:
sol=sol.assign(Label=result)
sol.head()

,Image,filename,Label
0,kc0yq3aoedc1b0v05gwp.jpg,/kaggle/input/leaf-disease-classification/sol/...,7
1,udxt71dqhpo3je6alnu3.jpg,/kaggle/input/leaf-disease-classification/sol/...,0
2,n05xwwbvd8t0k2s6vxim.jpg,/kaggle/input/leaf-disease-classification/sol/...,4
3,fe2udf0166l7e914798x.jpg,/kaggle/input/leaf-disease-classification/sol/...,3
4,royn55uoa0oc330cmivt.jpg,/kaggle/input/leaf-disease-classification/sol/...,4


In [20]:
sol=sol[['Label']]
sol.head()

,Label
0,7
1,0
2,4
3,3
4,4


In [ ]:
sol.to_csv('./ResNet50.csv',index=False)